## Case - Análise de dados da COVID-19 para o Estado de SP
##### Enunciado:
No site https://www.saopaulo.sp.gov.br/planos/simi/dados-abertos/ estão organizados dados de covid-19 para o Estado de SP.

Utilizando o arquivo CSV de microdados dos casos, construa um jupyter notebook para responder a essas perguntas:

1. Existe diferença de incidência de covid entre homens e mulheres?     
2. Faça um gráfico da mortalidade por faixas de idade       
3. Qual a doença pré-existente mais provável de se encontrar numa pessoa com covid?     
4. Baseado nesses dados, faça um modelo que estime a probabilidade da pessoa morrer, uma vez que está contaminada com covid, e considerando os inputs de idade, gênero e doenças pré-existentes

In [28]:
# imports utilizados:

import pandas as pd
import numpy as np
import statsmodels as sm
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools import eval_measures

In [59]:
# Importando e analisando brevemente o dataframe
df = pd.read_csv("C:\\VS\\projetos\\covid_sp\\20240120_Casos_e_obitos_ESP.csv", sep=";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6770928 entries, 0 to 6770927
Data columns (total 19 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Asma                     object 
 1   Cardiopatia              object 
 2   Data_Inicio_Sintomas     object 
 3   Diabetes                 object 
 4   Diagnostico_Covid19      object 
 5   Doenca_Hematologica      object 
 6   Doenca_Hepatica          object 
 7   Doenca_Neurologica       object 
 8   Doenca Renal             object 
 9   Genero                   object 
 10  Idade                    float64
 11  Imunodepressao           object 
 12  Municipio                object 
 13  Obesidade                object 
 14  Obito                    int64  
 15  Outros_Fatores_De_Risco  object 
 16  Pneumopatia              object 
 17  Puérpera                 object 
 18  Síndrome_De_Down         object 
dtypes: float64(1), int64(1), object(17)
memory usage: 981.5+ MB


In [60]:
df['Diagnostico_Covid19'].describe()
# Temos que todos os dados do dataset são de casos confirmados

count        6770928
unique             1
top       CONFIRMADO
freq         6770928
Name: Diagnostico_Covid19, dtype: object

## 1. Existe diferença de incidência de covid entre homens e mulheres?

In [10]:
df['Genero'].value_counts()

FEMININO      3714415
MASCULINO     3053061
INDEFINIDO       3442
IGNORADO            9
Name: Genero, dtype: int64

In [62]:
masc_per100 = (len(df.loc[df['Genero']=='MASCULINO'])/len(df))*100
fem_per100 = (len(df.loc[df['Genero']=='FEMININO'])/len(df))*100
fem_per100-masc_per100

9.767553280731974

In [81]:
Generos = ['MASCULINO','FEMININO']
Porcentagem_por_genero = [masc_per100, fem_per100]

fig = px.bar(y=Generos,x=Porcentagem_por_genero,)
fig.update_layout(title_text='Distribuição de infectados por COVID por gênero', xaxis_title='Porcentagem de infectados', yaxis_title='Gênero')
fig.show()

Podemos dizer que com uma diferença de 9,76%, a diferença é significativa usando como parâmetro 5%. Isso significa que em diferentes amostras, essa diferença também existirá.

Porém, estamos analisando a distribuição de gênero DADO os dados de infectados, mas o resultado pode ser diferente, ou não significativo, se:
1) Extendermos a análise para incidência de casos sobre toda a população de São Paulo; neste caso, teríamos (casos_confirmados_masc) / (população_masc) e o mesmo para a população feminina; se a população feminina for maior que a masculina em SP, a proporção pode ser menor.
2) Considerarmos que existe **endogeneidade**; no caso, como o dataset possui apenas casos confirmados de covid, existe um viés de seleção em toda amostra, porque fazer o teste para covid é uma escolha individual; dentro disso, pode-se considerar, por exemplo, que mulheres tem uma tendência maior a cuidar da saúde e procurar fazer testes.

Nesse sentido, a análise é fraca ou mesmo inválida para dizer se de fato há uma diferença de _incidência_ [efeito causal] de covid sobre diferentes gêneros.

## 2. Faça um gráfico da mortalidade por faixas de idade

Ao pensar em mortalidade por faixa de idade, é mais relevante o valor relativo do que o absoluto

In [13]:
df['Idade'].dropna().describe()
# Aparentemente, há alguns valores absurdos de idade, como pode-se ver em "max"
df['Idade'].dropna().sort_values()[-5:]
# É um único dado, que será removido

C:\Users\loren\AppData\Local\Temp\ipykernel_20336\1310401534.py:3: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df['Idade'].dropna().sort_values()[-5:]


6093435     122.0
2398720     122.0
4014312     125.0
73109       136.0
762764     2188.0
Name: Idade, dtype: float64

In [14]:
# Limpando o dataset
df = df.query('Idade < 140')
df['Idade'].dropna().sort_values().value_counts()

39.0     149407
38.0     149116
40.0     143375
37.0     142635
34.0     141097
          ...  
115.0         3
116.0         3
119.0         3
125.0         1
136.0         1
Name: Idade, Length: 125, dtype: int64

In [15]:
# Faixas de idade:
criancas = df.loc[df['Idade'] <= 12]['Idade']
adolescentes = df.loc[(df['Idade'] > 12) & (df['Idade'] <= 20)]['Idade']
adultos = df.loc[(df['Idade'] > 20) & (df['Idade'] <= 60)]['Idade']
idosos = df.loc[(df['Idade'] > 60)]['Idade']
faixas = [criancas, adolescentes, adultos, idosos]

# Valores absolutos de infectados e falecidos por covid
covid_faixas = [len(a) for a in faixas]
obitos_faixas = [len(b[(df['Obito'] == 1)]) for b in faixas]

# Proporção de óbitos por infectados por covid
pct_obitos = [(o / c)*100 for o,c in zip(obitos_faixas, covid_faixas)]
pct_obitos

[0.11106919820612768,
 0.07469585795229425,
 1.2427165018247837,
 12.030109064552306]

In [82]:
faixas=['Crianças', 'Adolescentes', 'Adultos', 'Idosos']

fig = px.bar(x=faixas, y=pct_obitos)
fig.update_layout(title_text='Porcentagem de óbitos por faixa de idade', xaxis_title='Faixas de idade', yaxis_title='Porcentagem de óbitos por infectados')
fig.show()

## 3. Qual a doença pré-existente mais provável de se encontrar numa pessoa com covid?

Como há diferentes valores para as observações em cada doença, pode ser interessante ver como elas aparecem ao remover a observação 'IGNORADO' para alguma doença

In [84]:
df2 = df.drop(df[(df['Cardiopatia']=='IGNORADO')].index)
df2

,Asma,Cardiopatia,Data_Inicio_Sintomas,Diabetes,Diagnostico_Covid19,Doenca_Hematologica,Doenca_Hepatica,Doenca_Neurologica,Doenca Renal,Genero,Idade,Imunodepressao,Municipio,Obesidade,Obito,Outros_Fatores_De_Risco,Pneumopatia,Puérpera,Síndrome_De_Down
14,IGNORADO,SIM,06/02/2021,IGNORADO,CONFIRMADO,IGNORADO,IGNORADO,IGNORADO,IGNORADO,FEMININO,43.0,IGNORADO,SÃO PAULO,IGNORADO,0,IGNORADO,IGNORADO,IGNORADO,IGNORADO
26,IGNORADO,SIM,06/04/2021,IGNORADO,CONFIRMADO,IGNORADO,IGNORADO,IGNORADO,IGNORADO,MASCULINO,71.0,IGNORADO,SÃO PAULO,IGNORADO,0,IGNORADO,IGNORADO,IGNORADO,IGNORADO
30,IGNORADO,SIM,10/03/2021,IGNORADO,CONFIRMADO,IGNORADO,IGNORADO,IGNORADO,IGNORADO,MASCULINO,68.0,IGNORADO,CAMPINAS,IGNORADO,0,IGNORADO,IGNORADO,IGNORADO,IGNORADO
33,IGNORADO,SIM,29/04/2022,IGNORADO,CONFIRMADO,IGNORADO,IGNORADO,IGNORADO,IGNORADO,FEMININO,50.0,IGNORADO,SÃO JOSÉ DO RIO PRETO,IGNORADO,0,IGNORADO,IGNORADO,IGNORADO,IGNORADO
39,IGNORADO,SIM,23/03/2021,IGNORADO,CONFIRMADO,IGNORADO,IGNORADO,IGNORADO,IGNORADO,MASCULINO,66.0,IGNORADO,SÃO PAULO,IGNORADO,0,IGNORADO,IGNORADO,IGNORADO,IGNORADO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6770883,NÃO,NÃO,10/11/2020,SIM,CONFIRMADO,NÃO,NÃO,NÃO,NÃO,MASCULINO,56.0,NÃO,SÃO JOSÉ DO RIO PRETO,NÃO,0,NÃO,NÃO,NÃO,NÃO
6770885,IGNORADO,SIM,27/04/2020,IGNORADO,CONFIRMADO,IGNORADO,IGNORADO,IGNORADO,IGNORADO,FEMININO,87.0,IGNORADO,SÃO VICENTE,IGNORADO,1,IGNORADO,IGNORADO,IGNORADO,IGNORADO
6770894,IGNORADO,SIM,01/08/2020,SIM,CONFIRMADO,IGNORADO,IGNORADO,IGNORADO,IGNORADO,FEMININO,80.0,IGNORADO,SÃO PAULO,IGNORADO,0,SIM,SIM,IGNORADO,IGNORADO
6770911,NÃO,SIM,06/06/2022,SIM,CONFIRMADO,NÃO,NÃO,NÃO,NÃO,MASCULINO,69.0,NÃO,CAMPINAS,NÃO,0,SIM,NÃO,NÃO,NÃO


Problema com o dataset: para determinadas observações, há variáveis (cada doença) observadas ('SIM' ou 'NÃO') enquanto nem todas as outras variáveis são observadas. Ou seja, apesar de haverem várias observações onde todas as doenças são 'IGNORADO', quando uma não é, não necessariamente as outras são; isso dificulta a análise.

Nesse caso, considerarei que há endogeneidade completa para observar os casos - ou seja, vou considerar que todos os casos de IGNORADO são equivalentes a um NÃO, e portanto, usarei os valores aboslutos da amostra.

A hipótese é razoável pois estamos trabalhando com dados de saúde, e é comum que casos só sejam investigados (nesse caso, com valor diferente de 'IGNORADO') apenas quando há suspeita ou tem-se confirmação prévia dessas doenças, sendo raro os casos em que doenças são investigadas quando não há sintoma ou preocupação relevante com tal doença.

In [85]:
doencas = ['Asma','Cardiopatia','Diabetes','Doenca_Hematologica','Doenca_Hepatica','Doenca_Neurologica','Doenca Renal','Imunodepressao','Obesidade','Outros_Fatores_De_Risco','Pneumopatia','Puérpera','Síndrome_De_Down']

qtd_doencas_total = [len(df[(df[a]== 'SIM')]) for a in doencas]

px.bar(x=doencas, y=qtd_doencas_total)

É perceptível que a doença mais comum observada em pacientes de covid é a **Cardiopatia**, seguida por **Diabetes**

## 4. Baseado nesses dados, faça um modelo que estime a probabilidade da pessoa morrer, uma vez que está contaminada com covid, e considerando os inputs de idade, gênero e doenças pré-existentes

Queremos construir um modelo que faça uma previsão para um valor binário: 1 para óbito e 0 para não-óbitos.

Existem vários modelos bons para predição de variáveis binárias, como: Logistic Regression, Random Forest, Decision Tree, Naive Bayes e Support Vector Machine (SVM), entre outros.

Considerando o escopo deste Case, usarei um Logit. Ainda, usarei duas bibliotecas diferentes: _scikit-learn_ e _statsmodels_.

In [117]:
# Limpando o dataframe
df = df.dropna().drop(columns=['Diagnostico_Covid19','Data_Inicio_Sintomas','Municipio'])

# Usarei 0 para NÃO e 1 para SIM; 2 e 3 para IGNORADO e INDEFINIDO, respectivamente.
for n in doencas:
    df[n].replace({'NÃO': 0, 'SIM': 1, 'IGNORADO': 2, 'INDEFINIDO':3}, inplace=True)
df['Genero'].replace({'MASCULINO':0, 'FEMININO':1, 'IGNORADO': 2, 'INDEFINIDO':3}, inplace=True)

df

,Asma,Cardiopatia,Diabetes,Doenca_Hematologica,Doenca_Hepatica,Doenca_Neurologica,Doenca Renal,Genero,Idade,Imunodepressao,Obesidade,Obito,Outros_Fatores_De_Risco,Pneumopatia,Puérpera,Síndrome_De_Down
0,2,2,2,2,2,2,2,0,69.0,2,2,0,2,2,2,2
1,2,2,2,2,2,2,2,1,60.0,2,2,0,2,2,2,2
2,2,2,2,2,2,2,2,0,58.0,2,2,0,2,2,2,2
3,2,2,2,2,2,2,2,1,45.0,2,2,0,2,2,2,2
4,2,2,2,2,2,2,2,1,42.0,2,2,0,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6770923,2,2,2,2,2,2,2,0,1.0,2,2,0,2,2,2,2
6770924,2,2,2,2,2,2,2,0,9.0,2,2,0,2,2,2,2
6770925,2,2,2,2,2,2,2,0,0.0,2,2,0,2,2,2,2
6770926,0,1,1,0,0,0,0,1,77.0,0,0,1,2,0,0,0


In [118]:
# Dividindo os dados em variáveis explicativas e explicada
X = df.loc[:, df.columns != 'Obito']
y = df['Obito']

# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

#### Modelo Logit com Scikit-Learn

In [92]:
# Modelo Logit com Scikit-Learn
logit = LogisticRegression(max_iter=200, solver='lbfgs')
logit.fit(X_train, y_train)
logit_y_pred = logit.predict(X_test)

In [138]:
acc_logit = round(logit.score(X_train, y_train) * 100, 2) # A função score() para Logit retorna o accuracy_score()
mse_logit = round(metrics.mean_squared_error(y_test, logit_y_pred)* 100, 2) # Mean Squared Error
precision_logit = round(metrics.precision_score(y_test, logit_y_pred)* 100, 2) # Precision Score

sklearn_metrics = [acc_logit, mse_logit, precision_logit]
sklearn_metrics

[97.17, 2.84, 47.84]

#### Modelo Logit com StatsModels

In [124]:
sm_logit = Logit(endog=list(y_train), exog=X_train)
logit_res = sm_logit.fit(method='lbfgs', maxiter=200)
sm_y_pred = logit_res.predict(X_test)

print(logit_res.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:              4889369
Model:                          Logit   Df Residuals:                  4889354
Method:                           MLE   Df Model:                           14
Date:                Thu, 30 May 2024   Pseudo R-squ.:                  0.3028
Time:                        00:01:18   Log-Likelihood:            -4.3513e+05
converged:                       True   LL-Null:                   -6.2413e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Asma                       -0.3950      0.019    -20.772      0.000      -0.432      -0.358
Cardiopatia                -1.1172      0.007   -165.956      0.000      -1.130      -1.104


In [135]:
# Função de predição do sm não está em binário, convertendo:
sm_y_pred = [round(n) for n in sm_y_pred.values]

In [140]:
# Como o StatsModels não tem funções suficientes, é necessário usar as funções do sklearn, mas com os dados preditos do modelo do sm
sm_acc_logit = round(metrics.accuracy_score(y_test, sm_y_pred)*100, 2) # Accuracy Score
sm_mse_logit = round(eval_measures.mse(y_test, sm_y_pred)*100, 2) # Mean Squared Error
precision_logit = round(metrics.precision_score(y_test, sm_y_pred)* 100, 2) # Precision Score

sm_metrics = [sm_acc_logit, sm_mse_logit, precision_logit]
sm_metrics

[96.84, 3.16, 38.83]

#### Comparando os modelos

In [141]:
pd.DataFrame({
    'Modelo':['sklearn','statsmodels'],
    'Accuracy':[sklearn_metrics[0],sm_metrics[0]],
    'MSE':[sklearn_metrics[1],sm_metrics[1]],
    'Precision':[sklearn_metrics[2],sm_metrics[2]]
})

,Modelo,Accuracy,MSE,Precision
0,sklearn,97.17,2.84,47.84
1,statsmodels,96.84,3.16,38.83


Ambos foram estimados pelo mesmo método (lbfgs, ou Limited-memory Broyden–Fletcher–Goldfarb–Shanno Algorithm), além da mesma limitação máxima de iterações - 200.

Contudo, o modelo do Scikit-Learn deu resultados mais precisos e bem mais eficientes (2 minutos conta quase 5 do modelo do sm), provavelmente por esepecificidades da biblioteca: Scikit-Learn é uma bibliteca mais especializada em previsão de modelos e Machine Learning do que StatsModels. 

Além disso, pode-se dizer que os resultados são satisfatórios como um todo, já que ambos tem mais de 95% de precisão. Em avaliações mais avançadas, seria interessante testar outros modelos, testar novos hiperparâmetros e avaliar questões como overfitting.

Dessa forma, eu usaria como modelo preditivo o >> logit.predict(x) para qualquer dataframe com novos dados de doenças pré-existentes para previsão de óbitos.